# <center>第3章 数学建模</center>
&emsp;&emsp;马尔可夫决策过程(Markov Decision Process, MDP)是强化学习的数学理论基础。通过马尔可夫决策过程以概率形式对强化学习任务进行建模，并对强化学习过程中出现的状态、动作、状态转移概率和奖赏等概念进行抽象表达，以实现对强化学习任务的求解，即得到最优策略，获得最大累积奖赏。

## 3.1  确定环境下扫地机器人任务的MDP数学建模。<br>
&emsp;&emsp;考虑图3.1所描述的确定环境MDP问题：一个扫地机器人，在躲避障碍物的同时，一方面需要到指定的位置收集垃圾，另一方面可以到指定位置给机器人电池充电。<br>
<center><img src='./image/图3.1.png' width='200' height='200'></center>
<center>图1&ensp;扫地机器人环境</center>
&emsp;&emsp;扫地机器人任务的MDP数学建模如下：<br>
&emsp;&emsp;（1）状态<br>
&emsp;&emsp;在该问题中，状态$s$用来描述机器人所在的位置，可以用2维向量表示。为了简化问题，将状态空间离散化为24个不同的状态（图3.1中，共25个小方格，除去$[3,3]$障碍物位置外，每个小方格作为一个状态），按照图3.1中序号出现的先后顺序，用集合表示为：<br>
$$\mathcal{S}^+=\{S_0:[1,1],S_1:[2,1],S_2:[3,1],...,S_{11}:[2,3],S_{13}:[4,3],...,S_{19}:[5,4],...,S_{23}:[4,5],S_{24}:[5,5]\}$$
&emsp;&emsp;其中，充电桩所在的位置为状态$S_0=[1,1]$，垃圾所在的位置为状态$S_{19}=[5,4]$。坐标[3,3]表示障碍物的位置，机器人不会到达这里，因此不作为状态。通常状态$S_0$和$S_{19}$为终止状态或吸收状态（absorbing state），即一旦机器人到达这两个状态之一，就不会再离开，情节（episode）结束。<br>
&emsp;&emsp;（2）动作<br>
&emsp;&emsp;动作$a$用来描述机器人运动的方向，可以用2维向量表示。为了简化问题，将动作空间离散化为上、下、左、右4个不同的动作，用集合表示为：<br>
$$\mathcal{A}=\{上:[0,1],下:[0,-1],左:[-1,0],右:[1,0]\}$$
&emsp;&emsp;对于不同的状态$s$，动作空间可能不同。例如，对于状态$S_0=[1,1]$，它的动作空间为：$\mathcal{A}(S_0)=\{[0,1],[1,0]\}$，即机器人处于该状态时，只能采取上、右动作，但因为$S_0=[1,1]$为吸入状态，无论采取哪个动作，都会保持原地不动；对于状态$S_9=[5,2]$，它的动作空间为：$\mathcal{A}(S_9)=\{[-1,0],[0,-1],[0,1]\}$，即机器人处于该状态时，不能采取向右的动作；对于坐标$[3,3]$周围的4个状态，动作空间中的4个动作都可以采用，但采取指向障碍物的动作时，会保持原地不动。<br>
&emsp;&emsp;（3）状态转移<br>
&emsp;&emsp;在确定环境下，状态转移可以用两种方式来表示：既可以映射为下一个状态，也可以映射为到达下一个状态的概率。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;映射为下一个状态：
$$f(s,a)=\begin{cases}
s+a,& s\neq[1,1]且s\neq[5,4]且s+a\neq[3,3] \\ 
s,& 其他
\end{cases} \tag{3.5}$$
&emsp;&emsp;$\blacktriangleright$&emsp;映射到达下一个状态的概率：
$$p(s,a,s')=\begin{cases}
1,& (s+a=s'且s+a\neq[3,3]) \\ & 或((s=[1,1]或s=[5,4]或s+a=[3,3])且s \neq s') \\
0,& 其他
\end{cases} \tag{3.6}$$
&emsp;&emsp;（4）奖赏<br>
&emsp;&emsp;到达状态$S_{19}=[5,4]$，机器人可以捡到垃圾，并得到+3的奖赏；到达状态$S_0=[1,1]$,机器人充电，并得到+1的奖赏；机器人采取动作向坐标$[3,3]$处移动时，会撞到障碍物，保持原地不动，并得到-10的奖赏；其他情况，奖赏均为0.特别是当机器人到达吸收状态后，无论采取什么动作，只能得到0的奖赏。对应的奖赏函数为：
$$r(s,a)= \begin{cases}
+3,& 如果s\neq[5,4]且s+a=[5,4] \\
+1,& 如果s\neq[1,1]且s+a=[1,1] \\
-10,& 如果s+a=[3,3] \\
0,& 其他
\end{cases} \tag{3.7}$$

&emsp;&emsp;在例3.1确定情况下扫地机器人任务中，采用的随机策略为：$\pi(a|S_i)=1/|\mathcal{A}(S_i)|$，$a\in \mathcal{A}(S_i)$，这里，$S_i$表示序号为$i$的状态，$|\mathcal{A}(S_i)|$表示状态$S_i$可以采取的动作数。在折扣率$\gamma=0.8$的情况下，求扫地机器人任务中每个状态的状态值。<br>
&emsp;&emsp;根据状态值函数贝尔曼方程，可以通过联立线性方程组的方法，求解得到：在对应策略$\pi$下，每个状态的状态值。针对处于状态$S_i$的扫地机器人，可以列出如式（3.21）的贝尔曼方程：
$$v_{\pi}(S_i)=\sum_{a\in \mathcal{A}(S_i)}\pi(a|S_i)p(S_i,a,s')(r(S_i,a)+\gamma v_{\pi}(s')) \tag{3.21}$$
&emsp;&emsp;式（3.21）中，确定环境下$p(S_i,a,s')=1$。状态值$v_{\pi}(s)$是对状态$s$的累积回报求期望。因为$S_0$和$S_{19}$是终止状态，在这两个状态下无论采取任何动作，都会保持原地不动，且累积奖赏为0，即这两个状态的状态值$v_{\pi}(S_0)$、$v_{\pi}(S_{19})$均为0。将每个状态的值函数 作为未知数，它们之间的关系满足贝尔曼方程，于是可以得到如下方程组：
$$\begin{cases}
v_{\pi}(S_1)=\frac{1}{3}\times(0+0.8\times v_{\pi}(S_6))+\frac{1}{3}\times(1+0.8\times 0)+\frac{1}{3}\times (0+0.8\times v_{\pi}(S_2)) \\
v_{\pi}(S_2)=\frac{1}{3}\times(0+0.8\times v_{\pi}(S_7))+\frac{1}{3}\times(1+0.8\times v_{\pi}(S_1))+\frac{1}{3}\times (0+0.8\times v_{\pi}(S_3)) \\
\vdots \\
v_{\pi}(S_{11})=\frac{1}{4}\times (0+0.8 \times v_{\pi}(S_{16})) + \frac{1}{4}\times (0+0.8 \times v_{\pi}(S_6)) + \frac{1}{4}\times (0+0.8 \times v_{\pi}(S_{10}) + \frac{1}{4}\times (-10+0.8 \times v_{\pi}(S_{11})) \\
\vdots \\
v_{\pi}(S_{13})=\frac{1}{4}\times (0+0.8 \times v_{\pi}(S_{18})) + \frac{1}{4}\times (0+0.8 \times v_{\pi}(S_8)) + \frac{1}{4}\times (-10+0.8 \times v_{\pi}(S_{13})) + \frac{1}{4}\times (0+0.8 \times v_{\pi}(S_{14})) \\
\vdots \\
v_{\pi}(S_{23})= \frac{1}{3}\times(0+0.8\times v_{\pi}(S_{18})) + \frac{1}{3}\times(0+0.8\times v_{\pi}(S_{22})) + \frac{1}{3}\times(0+0.8\times v_{\pi}(S_{24}))\\
v_{\pi}(S_{24})=\frac{1}{2}\times(3+0.8\times 0) + \frac{1}{2}\times(0+0.8\times v_{\pi}(S_{23}))
\end{cases}$$
&emsp;&emsp;求解方程组，得到状态值，如下图所示：<br>
<center><img src='./image/图3.8.png' width='275'></center>
<center>图3.8&ensp;基于等概率策略的确定环境扫地机器人任务的状态值</center>

In [1]:
import numpy as np

class sweeprobot():
    def __init__(self):
        # 状态空间
        self.S = [[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5]],
                  [[1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5]],
                  [[2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5]],
                  [[3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [4, 5]],
                  [[4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5]],
                  [[5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5]]]
        # 动作空间
        self.A = [[None, None], [-1, 0], [1, 0], [0, -1], [0, 1]]
        # 状态值
        self.V = [[None for i in range(6)] for j in range(6)]
        self.V[1][1] = 0
        self.V[5][4] = 0
        # 策略
        self.pi = None
        self.gamma = 0.8

    def reward(self, s, a):
        [truth1, truth2] = np.add(s, a) == [5, 4]
        [truth3, truth4] = np.add(s, a) == [1, 1]
        [truth5, truth6] = np.add(s, a) == [3, 3]
        if s != [5, 4] and (truth1 and truth2):
            return 3
        if s != [1, 1] and (truth3 and truth4):
            return 1
        if truth5 and truth6:
            return -10
        return 0

    def cal_coefficient(self):
        # 该函数用来计算出线性方程组的系数矩阵和向量值
        # 首先初始化一个25 * 25的系数矩阵和25个元素的向量
        coef_Martix = [[0 for i in range(25)] for j in range(25)]
        b = [0 for i in range(25)]
        for i in range(1, 6):
            for j in range(1, 6):
                # 判断是否是终止情况，如果是的话直接计算下一个
                [truth1, truth2] = [i == 5, j == 4]
                [truth3, truth4] = [i == 1, j == 1]
                [truth5, truth6] = [i == 3, j == 3]
                if truth1 and truth2:
                    continue
                if truth3 and truth4:
                    continue
                if truth5 and truth6:
                    continue
                # 计算当前状态下的动作数，以用于计算策略pi
                count_action = 0
                if i - 1 >= 1:
                    count_action += 1
                if i + 1 <= 5:
                    count_action += 1
                if j - 1 >= 1:
                    count_action += 1
                if j + 1 <= 5:
                    count_action += 1
                self.pi = 1 / count_action
                # 具体计算每一个状态值的函数
                b_value = 0
                coef_CurrentState = 0
                # 向上的情况
                if i - 1 >= 1:
                    b_value = b_value + self.pi * self.reward(self.S[i][j], self.A[1])
                    if i - 1 == 3 and j == 3:
                        coef_CurrentState = self.pi * self.gamma
                    else:
                        coef1 = self.pi * self.gamma
                        coef_Martix[(i - 1) * 5 + j - 1][((i - 1) - 1) * 5 + j - 1] = coef1
                # 向下的情况
                if i + 1 <= 5:
                    b_value = b_value + self.pi * self.reward(self.S[i][j], self.A[2])
                    if i + 1 == 3 and j == 3:
                        coef_CurrentState = self.pi * self.gamma
                    else:
                        coef2 = self.pi * self.gamma
                        coef_Martix[(i - 1) * 5 + j - 1][((i + 1) - 1) * 5 + j - 1] = coef2
                # 向左的情况
                if j - 1 >= 1:
                    b_value = b_value + self.pi * self.reward(self.S[i][j], self.A[3])
                    if j - 1 == 3 and i == 3:
                        coef_CurrentState = self.pi * self.gamma
                    else:
                        coef3 = self.pi * self.gamma
                        coef_Martix[(i - 1) * 5 + j - 1][(i - 1) * 5 + (j - 1) - 1] = coef3
                # 向右的情况
                if j + 1 <= 5:
                    b_value = b_value + self.pi * self.reward(self.S[i][j], self.A[4])
                    if j + 1 == 3 and i == 3:
                        coef_CurrentState = self.pi * self.gamma
                    else:
                        coef4 = self.pi * self.gamma
                        coef_Martix[(i - 1) * 5 + j - 1][(i - 1) * 5 + (j + 1) - 1] = coef4
                # 将左边的移项，所以系数为-1
                coef_Martix[(i - 1) * 5 + j - 1][(i - 1) * 5 + j - 1] = -1 + coef_CurrentState
                # 同理，将常数项移项需要乘-1
                b[(i - 1) * 5 + j - 1] = -1 * b_value
        # 因为状态[1,1]和状态[5,4]可以确定其状态值为0,状态[3,3]不存在，所以其实只需求22*22的矩阵和22个元素的向量值
        # 把矩阵和向量第[(1-1)*5+1-1]和[(5-1)*5+4-1]删除
        del coef_Martix[23]
        del b[23]
        del coef_Martix[12]
        del b[12]
        del coef_Martix[0]
        del b[0]
        # 把矩阵每一行的[(1-1)*5+1-1]和[(5-1)*5+4-1]和[(3-1)*5+3-1]删除
        for item in coef_Martix:
            del item[23]
            del item[12]
            del item[0]
        return coef_Martix, b

    def solve_equation(self, coef_Martix, b):
        A = np.array(coef_Martix)
        b = np.array(b)
        x = np.linalg.solve(A, b)
        x = list(x)
        for i in range(1, 6):
            for j in range(1, 6):
                [truth1, truth2] = [i == 5, j == 4]
                [truth3, truth4] = [i == 1, j == 1]
                [truth5, truth6] = [i == 3, j == 3]
                if truth1 and truth2:
                    continue
                if truth3 and truth4:
                    continue
                if truth5 and truth6:
                    continue
                self.V[i][j] = x.pop(0)

    def print_value(self):
        # 输出扫地机器人的状态值
        print('扫地机器人在随机策略下的状态值：')
        for i in range(1, 6):
            for j in range(1, 6):
                if self.V[j][6 - i] != None:
                    print('%.3f'%self.V[j][6 - i], end=" ")
                else:
                    print(self.V[j][6 - i], end=" ")
            print()

if __name__ == '__main__':
    sr = sweeprobot()
    A, b = sr.cal_coefficient()
    sr.solve_equation(A, b)
    sr.print_value()

扫地机器人在随机策略下的状态值：
-1.111 -1.359 -1.615 -0.329 1.368 
-1.417 -2.372 -4.369 -0.987 0.000 
-1.831 -4.716 None -3.987 -0.300 
-0.731 -2.162 -4.649 -2.160 -0.887 
0.000 -0.716 -1.772 -1.280 -0.867 


## 3.2 实验题
&emsp;&emsp;（编程）通过解方程组计算：HelloWord环境中，在折扣因子为$\gamma = 0.8$时，计算每个状态的状态值。
如图所示，共有25个状态，在每个状态下，都可以执行4个动作：北、南、西、东。奖赏设置：agent试图离开该区域，得到-1奖赏，并保持原地不动；在A点采取任何动作都转移到A’点，得到+10的奖赏；在B点采取任何动作都转移到B’点，得到+5的奖赏；其他情况奖赏为0。
<center>
<table>
    <tr>
        <th><img src='./image/GridWorld.png' width='300'></th>
    </tr>
</table>
</center>
<center>图3.12&ensp; Grid World示意图</center>


### 3.6.1 计算Hello World状态值代码如下：